In [88]:
from PIL import Image, ImageDraw,ImageFont,ImageChops

import sys
sys.path.append(r'/home/jupyter/reusable_code')
import google_api_functions as gaf

# Google Sheets Credentials
creds=gaf.Authenticate_Google(r'/home/jupyter/reusable_code/') # Return logged-in credentials

values,colourDf=gaf.read_google_sheets_as_rows('18t_E2ZBbFxxm32ApsajUMP1KGo--m2aqEO6xy4v4XvY','Sheet1',creds)
colourDictRGB={i['Colour']:(int(i['R']),int(i['G']),int(i['B'])) for i in colourDf[['Colour','R','G','B','Colour Palette']].to_dict('records') if i['Colour Palette']=='Overall'}
colourDictRGBInternal={i['Colour']:(int(i['R']),int(i['G']),int(i['B'])) for i in colourDf[['Colour','R','G','B','Colour Palette']].to_dict('records') if i['Colour Palette']=='Internal'}# and i['Colour'][-9:]!='Highlight']
#colourDf



# Create a blank canvas and write the text on it

In [112]:
def trim(im, border):
    bg = Image.new(im.mode, im.size, border)
    diff = ImageChops.difference(im, bg)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)
    
def Concat_Image_and_Text(image_to_merge,text_string,text_colour='black',savepath=None,
    # Specify canvass size
    canvasHeight=1000,
    canvasWidth=1000,
    # Specify how big the text box should be as a proportion of the canvass
    max_text_height_pc=0.2, # 20% of the height
    max_text_width_pc=1, # 100% of width
    textIndentSize_pc=0.05 # 5% of size 
):

    
    # Calculate how big the text box should be in absolute terms
    textIndentSize=textIndentSize_pc*canvasWidth
    max_text_height=max_text_height_pc*canvasHeight
    max_text_width=max_text_width_pc*(canvasWidth-(2*textIndentSize))

    # Draw a blank canvas
    img = Image.new('RGBA', (canvasWidth, canvasHeight), color = (0,0,0,0))
    d = ImageDraw.Draw(img)

    # Initialise Font
    fontSize=7
    fnt = ImageFont.truetype(r'/home/jupyter/reusable_code/Montserrat-Medium.ttf', fontSize)
    textboxWidth,textboxHeight=d.multiline_textsize(text_string, font=fnt, spacing=2) # See how large the text box would be to write the selected text in the selected space

    # Iteratively increase Text Size until it is too big
    while textboxWidth<max_text_width and textboxHeight<max_text_height:
        fontSize+=1
        fnt = ImageFont.truetype(r'/home/jupyter/reusable_code/Montserrat-Medium.ttf', fontSize)
        textboxWidth,textboxHeight=d.textsize(text_string, font=fnt, spacing=2)
        #print(textboxWidth,textboxHeight)

    # Identify the top left of the text box
    textLocY=canvasHeight-textboxHeight-textIndentSize
    textLocX=(canvasWidth/2)-(textboxWidth/2)

    picCanvasHeight=textLocY-1

    d.multiline_text((textLocX,textLocY), text_string, font=fnt, fill=text_colour,align='center') # Write the text in
    
    #### Add in an image above the text
    
    pic=Image.open(image_to_merge) # Open the image
    #print(pic.size)
    pic=trim(pic,0) # crop surrounding whitespace using function above
    #print(pic.size)
    
    pic = pic.convert("RGBA") # Convert to RGBA
    scaling_factor=min(canvasWidth/pic.size[0],picCanvasHeight/pic.size[1]) # Calculate the ratio needed to make the image fit the available space
    newsize=tuple(int(i*scaling_factor) for i in pic.size) # Apply that ratio to get a new image size
    pic =pic.resize(newsize) # Resize the image
    
    # Work out the top left coordinates of the image within the canvas, in order to centre the image
    picLocY=int((picCanvasHeight/2)-(pic.size[1]/2))
    picLocX=int((canvasWidth/2)-(pic.size[0]/2))
    img.paste(pic, (picLocX, picLocY),pic) # Paste the image in
    if savepath==None:
        savepath=image_to_merge.split('.')[0]+'_new.png'
  
    img.save(savepath,"PNG") # Save it
    


In [84]:
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon Trading_new.png', text_string='''Trading Report''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Reporting Trading.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon Play_new.png', text_string='''Viewing Dashboard''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Reporting Viewing.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon KPIs_new.png', text_string='''Customer Experience KPIs''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Reporting Customer Experience KPIs.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon Funnel_new.png', text_string='''Conversion Funnel''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Reporting Conversion Funnel.png')

In [85]:
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon Search BritBox_new.png', text_string='''Brand Tracker''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Reporting Brand Tracker.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon Search Competitors_new.png', text_string='''SVOD Market \nOverview''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Reporting SVOD Market Overview.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon Acquisition_new.png', text_string='''Joiners Survey''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Reporting Joiners Survey.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon Churn_new.png', text_string='''Leavers Survey''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Reporting Leavers Survey.png')

In [113]:
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Audience Segments_new.png', text_string='''SVOD Audience\nSegments''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight SVOD Audience Segments.png')

In [115]:
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/LoveIsland_brand_Packshot_720_1080_wkk03p7.jpeg', text_string='''Love Island 2021\nLoss of Exclusivity Impact''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight Love Island- 2021 Loss of Exclusivity.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Boris1_Spitting_Image_credit_AvalonBritBoxMark Harrison (2).jpeg', text_string='''Spitting Image\nLaunch Impact''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight Spitting Image Launch Impact.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon Churn Door_new.png', text_string='''Behaviour-driven\n Churn 2020''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight Churn Deep Dive.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon PiggyBank_new.png', text_string='''Lifetime\nValue 2020''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight Lifetime Value 2020.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon Chart_new.png', text_string='''Econometrics\nYear 1''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight Econometrics Year 1.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon Venn_new.png', text_string='''Content\nSegmentation''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight Content Segmentation.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Stripe vs iTunes_new.png', text_string='''Stripe vs iTunes\nChurn Mini-dive''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight Stripe vs iTunes Churn.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/BB_2021BAU_Marcella_Square_1080x1080.jpeg', text_string='''Drama Drops Impact\nJan 2021''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight Drama Drops.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon MegaPhone_new.png', text_string='''Messaging\nResearch 2020''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight Messaging Research 2020.png')
Concat_Image_and_Text(image_to_merge=r'/home/jupyter/Images/Icon LightBulb_new.png', text_string='''Editorial\nConcept Testing''',\
                      text_colour=colourDictRGBInternal['Union Blue'],savepath=r'/home/jupyter/Images/SiteIcon Insight Editorial Concept Testing.png')
